In [2]:
import pandas as pd
from sqlalchemy.types import VARCHAR, INTEGER, FLOAT
from sqlalchemy import text
from sqlalchemy import create_engine


In [3]:
# connect to the database
engine = create_engine('mssql+pyodbc://WagnerProdAGL1/Targit_DM?driver=ODBC+Driver+17+for+SQL+Server', echo=False, fast_executemany=True)

In [4]:
# Read OTIL excel
df_OTIF_Stage = pd.read_excel('Files/20231214 DBS and CMIS Grief.xlsx', sheet_name='CMIS Grief', skiprows=3)

### Logic to getting the calculations made


In [16]:
# # Assuming 'your_column_name' is the name of the column you want to pad with leading zeros
# df_OTIF_Stage['Store'] = df_OTIF_Stage['Store'].astype(str).str.zfill(2)

In [5]:
# Rename column 'Store' to 'Store_Number'
df_OTIF_Stage.rename(columns={'Store': 'Store_Number'}, inplace=True)

# Pad Store_Number with leading zeros if single digit
df_OTIF_Stage['Store_Number'] = df_OTIF_Stage['Store_Number'].astype(str).str.zfill(2)


In [8]:


# # Count 'part no' for the Aging column, transpose the values, then group by Store_Number
# result_df = df_OTIF_Stage.groupby(['Store_Number', 'Aging'])['Part No'].count().unstack(fill_value=0)

# # reset the index and have 'Store_Number' as a regular column:
# result_df = result_df.reset_index()



result_df = df_OTIF_Stage

result_df


In [10]:
# # Specify data types for each column
# dtype_mapping = {
#     'Store_Number': VARCHAR,
#     '30+ days': INTEGER,
#     '10-29 Days': INTEGER,
#     '0-9 Days': INTEGER,
# }

table_name = 'CMIS_Grief'

In [11]:
# Convert the DataFrame to a table in the dbo schema without explicit data types
result_df.to_sql('CMIS_Grief', con=engine, index=False, if_exists='replace', schema='dbo')

-1

In [35]:
# Execute the SQL queries
with engine.connect() as connection:
    # Create the final table
    connection.execute(text(sql_create_table_query))
    
    # Insert into final table
    connection.execute(text(sql_insert_query))
    
    # Truncate staging table
    connection.execute(text(sql_truncate_staging))

    # Commit the transaction
    connection.commit()


In [ ]:
# Write df_OTIL to SQL Server (replace 'final_table' with your actual table name)
# df_OTIL.to_sql('Parts_OTIL', con=engine, index=False, if_exists='replace')


# Write df_OTIL to Excel
df_OTIL.to_excel('OTIL_Testing.xlsx', index=False)